## [AIBoosted.dev](https://aiboosted.dev)
### Structured JSON LLM model output using Typescript, Langchain and OpenAI

In this example, we ask the LLM to review our text, and return the response as JSON, with the following data structure: `{ "reviewedText": "the reviewed text" }`

**To run this notebook locally, set up your own local Jupyter dev environmet: [Create an AI prototyping environment using Jupyter Lab IDE with Typescript, LangChain.js and Ollama for rapid AI prototyping](https://www.aiboosted.dev/p/install-jupyter-lab-deno-typescript-prototyping)**

You have to create a `.env` file with an OpenAI API key:

```
OPENAI_API_KEY=[your OpenAI API key]
```

The following code cell reads the `.env` file and sets the environment variables, so Langchain can use the OpenAI API key.

In [1]:
// auto-load .env
import "https://deno.land/std@0.215.0/dotenv/load.ts";

[Module: null prototype] {  }

In [2]:
import { ChatOpenAI, OpenAI } from "npm:@langchain/openai@~0.1";
import { DynamicStructuredTool } from "npm:@langchain/core@~0.2/tools";
import { PromptTemplate } from "npm:@langchain/core@~0.2/prompts";
import { z } from "npm:zod";

In [3]:
/**
 * Reviews and corrects the input text using OpenAI's GPT-4o model.
 * 
 * @param instruction - Instructions to be given to the model.
 * @param inputText - The text to be reviewed and corrected.
 * @returns - The reviewed and corrected text, or undefined if the response is invalid.
 */
async function reviewTextOpenAI(instruction: string, inputText: string): Promise<string | undefined> {
  // Create a prompt template using the provided instruction and input text
  const prompt = PromptTemplate.fromTemplate(
`{instruction}
---
{inputText}`);
  
  // Initialize the OpenAI chat model with specified options
  const llm = new ChatOpenAI({
    modelName: "gpt-4o", // Use the GPT-4 model
    verbose: false, // Disable verbose logging
  });

  // Define the schema for the model's output, it contains the reviewed text  
  const reviewedTextSchema = z.object({
    reviewedText: z.string().describe("The reviewed text.") // The reviewed text must be a string
  });
  type ReviewedTextSchema = z.infer<typeof reviewedTextSchema>; // Infer the TypeScript type from the Zod schema

  // We expect structured JSON output, we achieve this using OpenAI's function calling feature
  // More details: https://js.langchain.com/v0.2/docs/how_to/structured_output/
  const llmWithStructuredOutput = llm.withStructuredOutput(reviewedTextSchema, {
    method: "functionCalling",
    name: "withStructuredOutput"
  });  
  
  // Create a processing chain combining the prompt and the LLM
  // More details: https://js.langchain.com/v0.2/docs/how_to/sequence
  const chain = prompt.pipe(llmWithStructuredOutput);

  // Invoke the chain with the instruction and input text, and wait for the response
  const response = await chain.invoke({ instruction, inputText });

  // Return the reviewed text if present in the response, otherwise undefined  
  return response?.reviewedText;
}

// we specify the inputs for the prompt
const instruction = "Fix the grammar issues in the following text.";
const inputText = "How to stays relevant as the developer in the world of ai?";

// we show the reviewed text returned by the LLM
console.log(await reviewTextOpenAI(instruction, inputText));

How to stay relevant as a developer in the world of AI?
